String to binary to String

In [24]:
import re


def stringToAscii(s):
    return ''.join([str(ord(x)) for x in s.upper()])

def ascToString(s):
    chars = re.findall('..?', s)
    return ''.join([chr(int(x)) for x in chars])
 
asc = stringToAscii("AB c") 
print(asc)
print(ascToString(asc))


65663267
AB C


In [15]:
ord(' ')
chr(65)

'A'

In [11]:
'T'.ljust(4)

'T   '

In [26]:
a="0123456789"
print(a[0:5])
print(a[5:])

01234
56789


In [30]:
"""TECL,2021-11-09 10:00:00"""
def encode(date, symbol):
    dateText = date.replace("-", "")
    dateText = dateText.replace(" ", "")
    dateText = dateText.replace(":", "")
    symbolNumber = stringToAscii(symbol)
    return int(dateText + symbolNumber)

def decode(textNumber):
    text = str(textNumber)
    dateText = text[0:12]
    symbolNumber = text[14:]
    symbol = ascToString(symbolNumber)
    return f"{dateText} {symbol}"

s = encode("2021-11-09 10:00:00", "TECL")
print(f"encoded {s}")
s1 = decode(s)
print(f"decoded {s1}")

encoded 2021110910000084696776
decoded 202111091000 TECL


In [116]:
import pandas as pd

data = pd.read_csv('data/training/hourly_data.csv')

#drop currency rows
# data = data.drop(data[(data.symbol == 'USDCNY') | (data.symbol == 'USDEUR')].index)

# symbol_set = data.drop_duplicates(subset=['symbol'])['symbol'].tolist()
# print(f"total symbols: {len(symbol_set)}, {symbol_set}")


# data = data.drop_duplicates(subset=['symbol','time'], keep='last')
df1 = data.groupby(['time'])['symbol'].describe()[['count']]
print(len(data))
print(df1)

df2 = df1[df1['count']!=120]
print(df2)

67680
                    count
time                     
2021-10-11 09:35:00   120
2021-10-11 10:00:00   120
2021-10-11 11:00:00   120
2021-10-11 12:00:00   120
2021-10-11 13:00:00   120
...                   ...
2022-01-07 13:00:00   120
2022-01-07 14:00:00   120
2022-01-07 15:00:00   120
2022-01-07 16:00:00   120
2022-01-07 16:05:00   120

[564 rows x 1 columns]
Empty DataFrame
Columns: [count]
Index: []


In [113]:
# df3 = data[data.time == '2021-10-11 09:35:00']
# df4 = df3.groupby(['symbol'])['symbol'].describe()[['count']]
# print(df4[df4['count']>1])

df3.drop_duplicates(subset=['symbol', 'time'], inplace=True)
df3[df3.symbol=='IDXX']


/home/yao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,close,high,low,open,symbol,time,volume,date,hourTime,SMA10,SMA20,RSI,OBV,ADL,STOCH
54,615.49,617.0,615.49,617.0,IDXX,2021-10-11 09:35:00,7874.0,2021-10-11,09:35:00,182.877,154.979,98.834,188056597.0,-1.509708e+07,98.246


In [95]:
def check(check_time):
    day1 = data[data['time'] == check_time]
    symbols_day1 = day1.drop_duplicates(subset=['symbol'])['symbol'].tolist()
    # print(len(symbols_day1))
    notFound = [x for x in symbol_set if x not in symbols_day1]
    return notFound

In [ ]:
time_set = data.drop_duplicates(subset=['time'])['time'].tolist()
print(f"total times: {len(time_set)}, {time_set}")

In [98]:
# time = '2021-10-11 10:00:00'
for time in time_set:
    missedQuotes = check(time)
    for symbol in missedQuotes:
        df1 = data[(data['symbol'] == symbol)& (data['time']<time)].tail(1).copy()
        df1['time'] = time
        data = data.append(df1)
    missedQuotes = check(time)
    assert len(missedQuotes) == 0

In [114]:
data = data.round(3)
data = data.drop_duplicates(subset=['symbol','time'])
data.sort_values(by=['time','symbol'], inplace=True)
data.to_csv('data/training/hourly_data.csv', index=False)

In [66]:
symbol_set = data.drop_duplicates(subset=['symbol'])['symbol'].tolist()
print(len(symbol_set))

122


In [53]:
df = data[data['date']>='2022-01-01']
df.sort_values(by=['time','symbol'], inplace=True)
df

,close,high,low,open,symbol,time,volume,date,hourTime,SMA10,SMA20,RSI,OBV,ADL,STOCH
44,178.965,179.31,177.710,177.83,AAPL,2022-01-03 09:35:00,3342579.0,2022-01-03,09:35:00,177.144,175.490,66.849,6.319787e+08,7.456989e+08,94.358
608,134.880,135.70,134.760,135.41,ABBV,2022-01-03 09:35:00,257243.0,2022-01-03,09:35:00,133.668,129.819,72.246,5.487301e+08,5.667837e+08,87.074
1171,138.650,139.83,138.400,139.43,ABT,2022-01-03 09:35:00,193379.0,2022-01-03,09:35:00,139.192,136.677,60.802,5.925262e+08,5.598410e+08,66.415
1734,414.500,415.53,413.050,414.66,ACN,2022-01-03 09:35:00,86732.0,2022-01-03,09:35:00,406.728,392.293,63.859,6.044241e+08,5.548966e+08,80.028
2297,572.730,575.00,566.505,566.65,ADBE,2022-01-03 09:35:00,110794.0,2022-01-03,09:35:00,565.918,594.876,38.982,5.939367e+08,5.521441e+08,26.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64426,54.770,55.06,54.770,54.79,WFC,2022-01-07 16:05:00,638313.0,2022-01-07,16:05:00,50.608,49.598,69.639,7.673300e+08,4.529919e+07,95.526
64990,144.872,144.90,144.850,144.89,WMT,2022-01-07 16:05:00,393478.0,2022-01-07,16:05:00,143.309,142.439,59.205,7.928849e+08,5.308191e+07,87.305
65553,68.730,68.88,68.700,68.88,XOM,2022-01-07 16:05:00,724283.0,2022-01-07,16:05:00,64.016,62.536,69.921,1.030011e+09,5.954025e+07,97.326
66116,45.000,45.24,45.000,45.05,XPEV,2022-01-07 16:05:00,19656.0,2022-01-07,16:05:00,47.085,45.493,46.134,1.105254e+09,9.457593e+07,50.688


In [55]:
df.reset_index(inplace=True)
df[0:120]

,level_0,index,close,high,low,open,symbol,time,volume,date,hourTime,SMA10,SMA20,RSI,OBV,ADL,STOCH
0,0,44,178.965,179.310,177.710,177.83,AAPL,2022-01-03 09:35:00,3342579.0,2022-01-03,09:35:00,177.144,175.490,66.849,6.319787e+08,7.456989e+08,94.358
1,1,608,134.880,135.700,134.760,135.41,ABBV,2022-01-03 09:35:00,257243.0,2022-01-03,09:35:00,133.668,129.819,72.246,5.487301e+08,5.667837e+08,87.074
2,2,1171,138.650,139.830,138.400,139.43,ABT,2022-01-03 09:35:00,193379.0,2022-01-03,09:35:00,139.192,136.677,60.802,5.925262e+08,5.598410e+08,66.415
3,3,1734,414.500,415.530,413.050,414.66,ACN,2022-01-03 09:35:00,86732.0,2022-01-03,09:35:00,406.728,392.293,63.859,6.044241e+08,5.548966e+08,80.028
4,4,2297,572.730,575.000,566.505,566.65,ADBE,2022-01-03 09:35:00,110794.0,2022-01-03,09:35:00,565.918,594.876,38.982,5.939367e+08,5.521441e+08,26.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,64470,49.260,49.300,48.915,48.93,WFC,2022-01-03 09:35:00,1123257.0,2022-01-03,09:35:00,48.370,48.833,58.321,7.021082e+08,4.857856e+06,97.812
116,116,65034,144.205,144.570,143.860,144.00,WMT,2022-01-03 09:35:00,262883.0,2022-01-03,09:35:00,141.687,141.300,60.999,7.933589e+08,6.069253e+07,78.947
117,117,65597,61.660,61.664,61.210,61.24,XOM,2022-01-03 09:35:00,604935.0,2022-01-03,09:35:00,61.195,61.488,54.003,9.174502e+08,1.693332e+07,98.936
118,118,66160,50.560,51.500,50.540,51.12,XPEV,2022-01-03 09:35:00,814221.0,2022-01-03,09:35:00,46.061,45.607,58.542,1.127519e+09,1.087329e+08,90.596


In [57]:
df[120:240]

,level_0,index,close,high,low,open,symbol,time,volume,date,hourTime,SMA10,SMA20,RSI,OBV,ADL,STOCH
120,120,43,180.450,180.930,179.920,180.220,AAPL,2022-01-03 10:00:00,2401215.0,2022-01-03,10:00:00,177.144,175.490,66.849,6.319787e+08,7.456989e+08,94.358
121,121,607,133.820,134.340,133.650,134.060,ABBV,2022-01-03 10:00:00,126575.0,2022-01-03,10:00:00,133.668,129.819,72.246,5.487301e+08,5.667837e+08,87.074
122,122,1170,137.520,138.060,137.450,137.650,ABT,2022-01-03 10:00:00,87122.0,2022-01-03,10:00:00,139.192,136.677,60.802,5.925262e+08,5.598410e+08,66.415
123,123,1733,407.320,408.410,406.920,407.650,ACN,2022-01-03 10:00:00,26148.0,2022-01-03,10:00:00,406.728,392.293,63.859,6.044241e+08,5.548966e+08,80.028
124,124,2296,554.730,556.465,553.267,555.320,ADBE,2022-01-03 10:00:00,101889.0,2022-01-03,10:00:00,565.918,594.876,38.982,5.939367e+08,5.521441e+08,26.871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,65033,143.780,144.330,143.760,144.320,WMT,2022-01-03 10:00:00,114077.0,2022-01-03,10:00:00,141.687,141.300,60.999,7.933589e+08,6.069253e+07,78.947
236,236,65596,62.620,62.660,62.420,62.470,XOM,2022-01-03 10:00:00,413858.0,2022-01-03,10:00:00,61.195,61.488,54.003,9.174502e+08,1.693332e+07,98.936
237,237,66159,49.100,49.200,48.500,48.565,XPEV,2022-01-03 10:00:00,288822.0,2022-01-03,10:00:00,46.061,45.607,58.542,1.127519e+09,1.087329e+08,90.596
238,238,66722,178.165,178.590,177.240,178.170,ZM,2022-01-03 10:00:00,76530.0,2022-01-03,10:00:00,189.388,188.477,40.764,1.054336e+09,8.630802e+07,31.414
